In [ ]:
import yaml

dataset_info = {
'train': './train/images',
'val': './val/images',
'nc': 1 ,
'names': ['Human']
}

with open('./human_detection_dataset/data.yaml', 'w+') as f:
    doc = yaml . dump ( dataset_info , f, default_flow_style =None , sort_keys = False )

    

In [ ]:
from ultralytics import YOLO
import cv2
import timeit
dataset = "C:/Users/LGS/Desktop/STEP_7/human_det/human_detection_dataset"
# predict on a local image
model = YOLO("yolov8s.pt")
model.train(data="VisDrone.yaml", 
            epochs=100,
           save_period = 1) 



# 평가

### 화질 측정하기
- 화질을 5단위로 내리면서 confusion matrix 만들기
- csv에는 confusion matrix, precision, recall, f1-score가 담겨져 있어야함
- <b>weigth를 쓴 이유는 클래스의 불균형한 분포를 고려하기 위해서임<b>
- class간의 중요도가 동등하다고 판단되면 macro를 사용하면 된다\

### PSNR(Peak signal-to-noise ratio) 노이즈의 정도를 이용하여 화질 확인

#### blur처리의 커널 사이즈를 이용해서 검사

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import deque
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
import pandas as pd
import os
import timeit
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from collections import Counter, deque

In [ ]:
# 마우스 이벤트 콜백 함수
def draw_polygon(event, x, y, flags, param):
    global points
    if event == cv2.EVENT_LBUTTONDOWN:  # 왼쪽 버튼 클릭 시
        points.append((x, y))  # 클릭한 좌표를 리스트에 추가
        print(points)
        cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)  # 클릭한 좌표에 작은 원 표시
        if len(points) > 1:
            cv2.line(frame, points[-2], points[-1], (0, 0, 255), 2)  # 이전 점과 현재 점을 연결

def point_in_polygon(point, polygon):
    point = Point(point)
    polygon = Polygon(polygon)
    if polygon.contains(point):
        return "invade"
    else:
        return "person"
# 이미지 화질 낮추기
def blur_image(image, kernel_size):
    blurred_image = cv2.blur(image, (kernel_size, kernel_size))
    return blurred_image
    

In [ ]:
######## default modify ########
file_save_path = f"{os.getcwd()}/confusion_matrix/blur/"
video_save_path = f"{file_save_path}/video/"
csv_save_path = f"{file_save_path}/csv/"
os.makedirs(video_save_path, exist_ok= True)
os.makedirs(csv_save_path, exist_ok= True)



#video psnr 확인
frame_qulity = 100
psnr_values = []

# model path
model_path = "C:/Users/LGS/Desktop/STEP_7/human_det/runs/detect/train8/"


######## model start ########

        
#model
model = YOLO(f"{model_path}/weights/best.pt")

# 비디오 로드
video_path = 'C:/Users/LGS/Desktop/STEP_7/human_det/test/human_invade.mp4'
frame_qulity = 1
Cnt = []
flag = 0
ground_truth_labels = []
df = pd.DataFrame(columns=['count','0','invade', 'person'])
ground_truth_label = 0

In [ ]:
for fq in range(frame_qulity, 30,1):
    df_con = pd.DataFrame(columns=['invade', 'person'])
    count = 0
    
    # 클래스 레이블 초기화
    predicted_labels = []
    #video 가져오기
    cap = cv2.VideoCapture(video_path)
    # 비디오 속성 가져오기
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    #video save
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_path = f'{video_save_path}/result_{fq}.mp4'  # 저장할 영상 파일 경로로 변경해주세요
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    points = []  # 클릭한 좌표들을 저장할 리스트
    
    cv2.namedWindow(f'{fq}')
    cv2.setMouseCallback(f'{fq}', draw_polygon)
    ######## 폴리곤 좌표 고정 ########
#     points = [(218, 330), (476, 718), (977, 482), (972, 220), (230, 187)] 
    invade_check = deque(maxlen=3)
    
    
    while True:
        count +=1
        invade_frame = 0
        ret, frame = cap.read()
        if not ret:
            break
    
        frame_copy = frame.copy()  # 원본 영상의 복사본 생성
        # 기존 이미지와 psnr 비교
        frame_degrade = blur_image(frame, fq)
        psnr = cv2.PSNR(frame_copy, frame_degrade)
        psnr_values.append(psnr)
        
        # 다각형 그리기
        mask = np.zeros_like(frame_degrade[:, :, 0])
        if len(points) > 1:
            points_arr = np.array(points)
            # cv2.fillPoly(mask, [points_arr], color=(255))  # 마스크 생성
            
            # frame_degrade[mask == 0] = 0 # 다각형 외부를 검은색으로 처리
            cv2.polylines(frame_degrade, [points_arr.astype(np.int32)], isClosed=True, color=(0, 0, 255), thickness=2)  # 다각형 테두리를 빨간색으로 그림
    
        results = model.predict(frame_degrade, verbose=False, conf=0.7)[0] # 모델을 예측함 ; 예측률 70% 이상이 아니면 예측 continue
        boxes = results.boxes.cpu().numpy()
        # polygon이 형성되었을 때 작용하는 코드
        
        if len(points) > 3:
            class_check = []
            for box in boxes:

                r = box.xyxy[0].astype(int)
                ct = box.xywh[0].astype(int)
                text_position = (r[0], r[1] - 10)
                class_name = point_in_polygon(ct[:2], points)
                cv2.rectangle(frame_degrade, r[:2], r[2:], (255, 255, 255), 2)
                confi = np.round(box.conf[0],4)
                cv2.putText(frame_degrade, f"{class_name}/ {confi:.3f}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2, cv2.LINE_AA)
                class_check.append(class_name)
                predicted_labels.append(class_name)
                
        else:
            class_check = []
            for box in boxes:

                r = box.xyxy[0].astype(int)
                ct = box.xywh[0].astype(int)
                text_position = (r[0], r[1] - 10)
                class_name = 'person'
                cv2.rectangle(frame_degrade, r[:2], r[2:], (255, 255, 255), 2)
                confi = np.round(box.conf[0],4)
                cv2.putText(frame_degrade, f"{class_name}/ {confi:.4f}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2, cv2.LINE_AA)
                class_check.append(class_name)
                predicted_labels.append(class_name)
        if len(results.boxes) == 0:
            predicted_labels.append('0')
    
        class_counts = Counter(class_check)
        if 'invade' in class_check :
            invade_frame = 'invade'
        invade_check.append(invade_frame)
        # Ground Truth 레이블 수집
        ############## 추후 여기에 제일 처음에 화질이 가장 좋은 모델을 불러오려고 함
        # 1보다 작을 땐 ground_truth_label를 지정함
        if fq == 1:
            ground_truth_label = predicted_labels
            
        # Visualize the results on the frame
        cv2.imshow(f'{fq}', frame_degrade)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('r'):  # 'r' 키를 누르면 다각형 그리기 종료
            points = []
            break
        # 프레임마다 만들어지는 데이터
        df_counter = pd.DataFrame.from_dict(Counter(class_check), orient='index', columns=['Count']).T
        df_counter = df_counter.reset_index(drop=True)

        df_con = pd.concat([df_con, df_counter], axis = 0)
        
        # 출력 영상에 결과 추가
        out.write(frame_degrade)
    ##### 데이터 저장 #####

    cd = Counter(predicted_labels)
    df_count = pd.DataFrame({'count' : [count],
                            'PSNR': [sum(psnr_values)/len(psnr_values)]})
    df3 = pd.concat([df_count], axis = 1)
    df = pd.concat([df,df3],axis = 0)
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    counter_csv = f'{csv_save_path}/fram_quality_{fq}.csv'  # 저장할 CSV 파일 경로로 변경해주세요
    df_con = df_con.fillna(0)
    df_con.to_csv(counter_csv, index = False)
    

# 결과를 CSV 파일로 저장
while os.path.exists(f'{video_save_path}/result_{vid}.mp4') :
    vid += 1
output_csv = f'{csv_save_path}/result_v{vid}.csv'  # 저장할 CSV 파일 경로로 변경해주세요
df.to_csv(output_csv, index=False)